<a href="https://colab.research.google.com/github/arcarrion/hsespecialization_course02/blob/master/02_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                    title                                              size  lastUpdated          downloadCount  
-----------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
anmolkumar/health-insurance-cross-sell-prediction      Health Insurance Cross Sell Prediction 🏠 🏥          6MB  2020-09-11 18:39:31            779  
shivan118/healthcare-analytics                         Healthcare Analytics                                2MB  2020-09-13 17:40:05            467  
Cornell-University/arxiv                               arXiv Dataset                                     885MB  2020-09-14 16:54:47           2831  
nipunarora8/age-gender-and-ethnicity-face-data-csv     AGE, GENDER AND ETHNICITY (FACE DATA) CSV          63MB  2020-09-02 13:46:38            261  
datatattle/c

In [1]:
#! kaggle competitions download -h
#! unzip -h

In [4]:
! mkdir rawdata
! kaggle competitions download -c competitive-data-science-predict-future-sales -p rawdata
! unzip 'rawdata/*.zip' -d rawdata
! rm rawdata/*.zip

mkdir: cannot create directory ‘rawdata’: File exists
  0% 0.00/2.91k [00:00<?, ?B/s]
100% 2.91k/2.91k [00:00<00:00, 5.59MB/s]
  0% 0.00/368k [00:00<?, ?B/s]
100% 368k/368k [00:00<00:00, 52.1MB/s]
  0% 0.00/3.49k [00:00<?, ?B/s]
100% 3.49k/3.49k [00:00<00:00, 3.30MB/s]
 98% 13.0M/13.3M [00:00<00:00, 40.6MB/s]
100% 13.3M/13.3M [00:00<00:00, 52.6MB/s]
  0% 0.00/468k [00:00<?, ?B/s]
100% 468k/468k [00:00<00:00, 150MB/s]
  0% 0.00/1.02M [00:00<?, ?B/s]
100% 1.02M/1.02M [00:00<00:00, 137MB/s]
Archive:  rawdata/items.csv.zip
  inflating: rawdata/items.csv       

Archive:  rawdata/sales_train.csv.zip
  inflating: rawdata/sales_train.csv  

Archive:  rawdata/test.csv.zip
  inflating: rawdata/test.csv        

Archive:  rawdata/sample_submission.csv.zip
  inflating: rawdata/sample_submission.csv  

4 archives were successfully processed.


In [5]:
print('################################################################# Item categories')
! head rawdata/item_categories.csv
print('################################################################# items')
! head rawdata/items.csv
print('################################################################# sales train')
! head rawdata/sales_train.csv
print('################################################################# shops')
! head rawdata/shops.csv
print('\n\n################################################################# test')
! head rawdata/test.csv
print('################################################################# sample submission')
! head rawdata/sample_submission.csv

################################################################# Item categories
item_category_name,item_category_id
PC - Гарнитуры/Наушники,0
Аксессуары - PS2,1
Аксессуары - PS3,2
Аксессуары - PS4,3
Аксессуары - PSP,4
Аксессуары - PSVita,5
Аксессуары - XBOX 360,6
Аксессуары - XBOX ONE,7
Билеты (Цифра),8
################################################################# items
item_name,item_id,item_category_id
! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D,0,40
"!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]",1,76
***В ЛУЧАХ СЛАВЫ   (UNV)                    D,2,40
***ГОЛУБАЯ ВОЛНА  (Univ)                      D,3,40
***КОРОБКА (СТЕКЛО)                       D,4,40
***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ  (UNI)             D,5,40
***УДАР ПО ВОРОТАМ (UNI)               D,6,40
***УДАР ПО ВОРОТАМ-2 (UNI)               D,7,40
***ЧАЙ С МУССОЛИНИ                     D,8,40
################################################################# sales train
date,date_block_num,shop_id,item_id

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [10]:
sales_rawdata = pd.read_csv('rawdata/sales_train.csv')


<bound method NDFrame.head of                date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0        02.01.2013               0       59    22154      999.00           1.0
1        03.01.2013               0       25     2552      899.00           1.0
2        05.01.2013               0       25     2552      899.00          -1.0
3        06.01.2013               0       25     2554     1709.05           1.0
4        15.01.2013               0       25     2555     1099.00           1.0
...             ...             ...      ...      ...         ...           ...
2935844  10.10.2015              33       25     7409      299.00           1.0
2935845  09.10.2015              33       25     7460      299.00           1.0
2935846  14.10.2015              33       25     7459      349.00           1.0
2935847  22.10.2015              33       25     7440      299.00           1.0
2935848  03.10.2015              33       25     7460      299.00           1.0

[2935849 

In [12]:
print(sales_rawdata.head())
print(sales_rawdata.dtypes)

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object


In [14]:
sales_noreturns = sales_rawdata.loc[sales_rawdata.item_cnt_day>=0, :]

In [17]:
sales_noreturns_grouped = sales_noreturns.groupby(['date_block_num','shop_id','item_id']).aggregate({'item_cnt_day':sum}).reset_index()
display(sales_noreturns_grouped)

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
...,...,...,...,...
1608221,33,59,22087,6.0
1608222,33,59,22088,2.0
1608223,33,59,22091,1.0
1608224,33,59,22100,1.0


In [43]:
median_sales = sales_noreturns_grouped.groupby(['shop_id','item_id']).aggregate({'item_cnt_day':'median'}).reset_index()
display(median_sales)
median_sales = sales_noreturns.loc[sales_noreturns.date_block_num==(34-12), :]
display(median_sales)

,shop_id,item_id,item_cnt_day
0,0,30,31.0
1,0,31,11.0
2,0,32,8.0
3,0,33,3.0
4,0,35,7.5
...,...,...,...
424093,59,22154,1.0
424094,59,22155,1.0
424095,59,22162,1.0
424096,59,22164,1.5


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2106209,19.11.2014,22,37,3460,1299.0,3.0
2106210,03.11.2014,22,33,4615,399.0,1.0
2106211,01.11.2014,22,33,4870,999.0,1.0
2106212,04.11.2014,22,33,4870,999.0,1.0
2106213,22.11.2014,22,33,4870,999.0,1.0
...,...,...,...,...,...,...
2192632,26.11.2014,22,42,59,249.0,1.0
2192633,07.11.2014,22,42,59,249.0,1.0
2192634,16.11.2014,22,42,77,149.0,1.0
2192635,10.11.2014,22,42,32,149.0,1.0


In [44]:
test_data = pd.read_csv('rawdata/test.csv')
median_sales_test = pd.merge(test_data, median_sales, how='left', on=['shop_id', 'item_id'])
display(median_sales_test)
median_sales_test.loc[pd.isnull(median_sales_test.item_cnt_day), 'item_cnt_day'] = 0
#median_sales_test.item_cnt_day = np.round(median_sales_test.item_cnt_day)
display(median_sales_test)

,ID,shop_id,item_id,date,date_block_num,item_price,item_cnt_day
0,0,5,5037,29.11.2014,22.0,2599.0,1.0
1,1,5,5320,NaN,NaN,NaN,NaN
2,2,5,5233,NaN,NaN,NaN,NaN
3,3,5,5232,NaN,NaN,NaN,NaN
4,4,5,5268,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
239176,214195,45,18454,NaN,NaN,NaN,NaN
239177,214196,45,16188,NaN,NaN,NaN,NaN
239178,214197,45,15757,NaN,NaN,NaN,NaN
239179,214198,45,19648,NaN,NaN,NaN,NaN


,ID,shop_id,item_id,date,date_block_num,item_price,item_cnt_day
0,0,5,5037,29.11.2014,22.0,2599.0,1.0
1,1,5,5320,NaN,NaN,NaN,0.0
2,2,5,5233,NaN,NaN,NaN,0.0
3,3,5,5232,NaN,NaN,NaN,0.0
4,4,5,5268,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...
239176,214195,45,18454,NaN,NaN,NaN,0.0
239177,214196,45,16188,NaN,NaN,NaN,0.0
239178,214197,45,15757,NaN,NaN,NaN,0.0
239179,214198,45,19648,NaN,NaN,NaN,0.0


In [45]:
sample_submission = median_sales_test.loc[:, ['ID','item_cnt_day']]
sample_submission.columns = ['ID', 'item_cnt_month']
display(sample_submission)

,ID,item_cnt_month
0,0,1.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
239176,214195,0.0
239177,214196,0.0
239178,214197,0.0
239179,214198,0.0


In [46]:
sample_submission.to_csv('submission.csv', header=True, index=False)

In [47]:
! kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m 'Testing submission'

100% 2.39M/2.39M [00:00<00:00, 6.86MB/s]
403 - Your team has used its submission allowance (5 of 5). This resets at midnight UTC (8.6 hours from now).
